# Compute confidence intervals for predictions on test set using CV+
Created by Ivan Lima on Sun Feb  5 2023 13:06:58 -0500

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, datetime, warnings
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

Last updated on Mon Feb  6 22:03:57 2023


In [2]:
import sns_settings
sns.set_context('paper')
pd.options.display.max_columns = 50
warnings.filterwarnings('ignore')

## Load DIC bottle data & select features & target

In [3]:
df_bottle_dic = pd.read_csv('data/bottle_data_DIC_prepared.csv', parse_dates=['Date'],
                            index_col=0, na_values=['<undefined>',-9999.])
df_bottle_dic = df_bottle_dic.loc[df_bottle_dic.Oxygen_flag.isin([2, 6])]
df_bottle_dic = df_bottle_dic.loc[df_bottle_dic.Oxygen.notnull()]
df_bottle_dic['log_Chl'] = np.log(df_bottle_dic.Chl)
df_bottle_dic['log_KD490'] = np.log(df_bottle_dic.KD490)

features = ['Depth', 'Temperature', 'Salinity', 'Oxygen', 'pCO2_atm', 'ADT', 'SST_hires', 'log_KD490']
target = ['DIC']

suffix = 'all_vars'

In [4]:
# df_bottle_dic = pd.read_csv('data/bottle_data_DIC_prepared.csv', parse_dates=['Date'],
#                             index_col=0, na_values=['<undefined>',-9999.])
# df_bottle_dic['log_Chl'] = np.log(df_bottle_dic.Chl)
# df_bottle_dic['log_KD490'] = np.log(df_bottle_dic.KD490)

# features = ['Depth', 'Temperature', 'Salinity', 'pCO2_atm', 'ADT', 'SST_hires', 'log_KD490']
# target = ['DIC']

# suffix = 'noO2'

In [5]:
# df_bottle_dic = pd.read_csv('data/bottle_data_DIC_prepared.csv', parse_dates=['Date'],
#                             index_col=0, na_values=['<undefined>',-9999.])
# df_bottle_dic['log_Chl'] = np.log(df_bottle_dic.Chl)
# df_bottle_dic['log_KD490'] = np.log(df_bottle_dic.KD490)

# features = ['Depth', 'Temperature', 'Salinity', 'pCO2_atm']
# target = ['DIC']

# suffix = 'nosat'

## Split data into training and test sets

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score

data = df_bottle_dic[features + target + ['Season']].dropna()

X = data[features].values
y = data[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=data.Season.values, random_state=77)
X.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3970, 8), (2977, 8), (993, 8), (2977, 1), (993, 1))

## Compute confidence intervals using CV+

In [7]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold
import time

keras.utils.set_random_seed(42) # make things reproducible
n_hidden = 256 # number of nodes in hidden layers
alpha=0.01

model = keras.models.Sequential([
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_hidden, input_shape=X_train.shape[1:]),
    keras.layers.LeakyReLU(alpha=alpha),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_hidden),
    keras.layers.LeakyReLU(alpha=alpha),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(y_train.shape[1])
])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam())

score_vals = []       # store score values
y_test_pred_list = [] # store predictions
resid = []            # store residuals

n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
start = time.time()

for k, (train_idx, test_idx) in enumerate(kf.split(X_train)):
    X_tr, X_te = X_train[train_idx], X_train[test_idx]
    y_tr, y_te = y_train[train_idx], y_train[test_idx]
    history = model.fit(X_tr, y_tr, epochs=700, verbose=0, validation_split=0.2, callbacks=[early_stopping_cb])
    y_pred = model.predict(X_test) # predictons on test set
    y_test_pred_list.append(y_pred.ravel())
    resid.append((y_te - model.predict(X_te)).ravel()) # residuals on sub test set
    score = r2_score(y_test, y_pred)
    score_vals.append(score)
    print('Fold {:02}/{} test set R squared: {:.3f}'.format(k+1, n_splits, score))

end = time.time()

scores = np.array(score_vals)
print('\nBest R squared:  {:.3f}'.format(scores.max()))
print('Worst R squared: {:.3f}'.format(scores.min()))
print('Mean R squared:  {:.3f}'.format(scores.mean()))

print('\nExecution time = {:.2f} minutes'.format((end-start)/60.))

2023-02-06 22:04:00.071045: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Fold 01/10 test set R squared: 0.962
Fold 02/10 test set R squared: 0.963
Fold 03/10 test set R squared: 0.963
Fold 04/10 test set R squared: 0.963
Fold 05/10 test set R squared: 0.964
Fold 06/10 test set R squared: 0.965
Fold 07/10 test set R squared: 0.961
Fold 08/10 test set R squared: 0.964
Fold 09/10 test set R squared: 0.966
Fold 10/10 test set R squared: 0.964

Best R squared:  0.966
Worst R squared: 0.961
Mean R squared:  0.964

Execution time = 1.47 minutes


In [8]:
percentile = 0.95
cv_preds = np.array(y_test_pred_list).transpose()
residuals = np.concatenate([x for x in resid])
ci = np.quantile(residuals, percentile)
if ci > 0:
    lower_bound = [np.quantile(cv_preds[n] - ci, percentile) for n in range(cv_preds.shape[0])]
    upper_bound = [np.quantile(cv_preds[n] + ci, percentile) for n in range(cv_preds.shape[0])]
else:
    lower_bound = [np.quantile(cv_preds[n] + ci, percentile) for n in range(cv_preds.shape[0])]
    upper_bound = [np.quantile(cv_preds[n] - ci, percentile) for n in range(cv_preds.shape[0])]

df_ci = pd.DataFrame({'DIC_observed':y_test.ravel(), 'lower_bound': lower_bound, 'upper_bound': upper_bound})

outfile = 'results/ci{:03d}_{}.csv'.format(int(percentile * 100), suffix)
print('\nwriting {}'.format(outfile))
df_ci.to_csv(outfile)


writing results/ci095_all_vars.csv


In [9]:
print(
    df_ci.loc[df_ci.DIC_observed < df_ci.lower_bound].shape[0],
    df_ci.loc[df_ci.DIC_observed > df_ci.upper_bound].shape[0]
)
print(ci)

82 33
19.415566406249933


In [10]:
# df_ensemble = pd.read_csv('results/ensemble_preds_dic_all_vars.csv', index_col=0)
# df = df_ensemble.drop('DIC_observed', axis=1)
# ensemble_preds = df.values

# lower_bound = [np.quantile(ensemble_preds[n], 0.05) for n in range(ensemble_preds.shape[0])]
# upper_bound = [np.quantile(ensemble_preds[n], 0.95) for n in range(ensemble_preds.shape[0])]
# df_ci2 = pd.DataFrame({'DIC_observed':df_ensemble.DIC_observed, 'lower_bound': lower_bound, 'upper_bound': upper_bound})

# df_ci2.loc[df_ci2.DIC_observed < df_ci2.lower_bound].shape, df_ci2.loc[df_ci2.DIC_observed > df_ci2.upper_bound].shape